# Getting data into a database

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy import create_engine
import ohio.ext.pandas
import time

## Let's start with the raw data

We'll use the data from donorschoose projects that comes in a csv file

In [2]:
datafile='/mnt/data/projects/food_inspections/data/projects.csv'

## Let's try putting it in our postgres database in a few different ways and figure out pros and cons

### Option 1: Use pandas to_sql

#### 1. Create schema in the postgres database

You can use psql or dbeaver

```create schema donorschoose```

#### 2. Load it in a paindas dataframe

In [3]:
df=pd.read_csv(datafile)


#### 3. Get postgres credentials and connection

In [4]:
engine=create_engine("postgresql:///food-inspections")


#### 4. Copy the dataframe to postgres (to the right schema and tablename)

In [5]:
df.to_sql(schema='donorschoose',name='projects',con=engine,if_exists='replace',index=False)

97

#### Let's talk about what happened and why it was so slow

...




### Option 2: Use the native COPY command from postgres

#### 1. Create schema in the postgres database (if you already did it earlier, you don't need to do it again)

You can use psql or dbeaver

```create schema donorschoose```

#### 2. Create table to copy data into

Here you can choose to 
1. put all the columns as varchars/strings
or 
2. get the right column types


You can also choose to:
1. Write the create table statement by hand
or 
2. Use a tool like csvkit to infer types and generate the create statement. 
```head -n 10000 | csvsql -i postgresql```


You can use psql or dbeaver to run it

```
create table donorschoose.projects
(projectid varchar(10000),
teacher_acctid varchar(10000),
 schoolid varchar(10000),
 school_ncesid varchar(10000),
 school_latitude varchar(10000),
 school_longitude varchar(10000),
 school_city varchar(10000),
 school_state varchar(10000),
 school_zip varchar(10000),
 school_metro varchar(10000),
 school_district varchar(10000),
 school_county varchar(10000),
 school_charter varchar(10000),
 school_magnet varchar(10000),
 school_year_round varchar(10000),
 school_nlns varchar(10000),
 school_kipp varchar(10000),
 school_charter_ready_promise varchar(10000),
 teacher_prefix varchar(10000),
 teacher_teach_for_america varchar(10000),
 teacher_ny_teaching_fellow varchar(10000),
 primary_focus_subject varchar(10000),
 primary_focus_area varchar(10000),
 secondary_focus_subject varchar(10000),
 secondary_focus_area varchar(10000),
 resource_type varchar(10000),
 poverty_level varchar(10000),
 grade_level varchar(10000),
 fulfillment_labor_materials varchar(10000),
 total_price_excluding_optional_support varchar(10000),
 total_price_including_optional_support varchar(10000),
 students_reached varchar(10000),
 eligible_double_your_impact_match varchar(10000),
 eligible_almost_home_match varchar(10000),
 date_posted varchar(10000)
 );
```



#### 3. Use the postgres COPY command to copy thew csv into the table you just created

You can use psql for this
```
food-inspections> \copy donorschoose.project from '/mnt/data/projects/food_inspections/data/projects.csv' header csv;
```

### Option 3: Use ohio, a python package we (thanks Jesse London) developed that combines the best of the two worlds

#### 1. Create schema in the postgres database (you don't need to do it again if you already did this)

You can use psql or dbeaver

```create schema donorschoose```

#### 2. Load it in a paindas dataframe

In [3]:
df=pd.read_csv(datafile)


#### 3. Copy it to the postgres databse using ohio pandas extension

In [6]:
with engine.connect() as conn:
    with conn.begin():
        df.pg_copy_to(schema='donorschoose',name='projects',con=conn, index=False, if_exists='replace')

### If you want to time how long each option takes, you can use time().

In [8]:
start_time = time.time()
df.to_sql(schema='donorschoose',name='projects',con=engine,if_exists='replace',index=False)
print("COPY duration: {} seconds".format(time.time() - start_time))

COPY duration: 86.23058223724365 seconds


In [9]:
start_time = time.time()
with engine.connect() as conn:
    with conn.begin():
        df.pg_copy_to(schema='donorschoose',name='projects',con=conn, index=False, if_exists='replace')
print("COPY duration: {} seconds".format(time.time() - start_time))

COPY duration: 10.623504400253296 seconds


### Using smaller chunks instead of loading it in one go if you data file is too large for pandas and the RAM you have

In [10]:
with engine.connect() as conn:
    with conn.begin():
        for df in pd.read_csv(datafile,  chunksize=1000000):
            df.pg_copy_to(schema='donorschoose', name='projects',con=conn,index=False, if_exists='append')

## Check to see if the data actually got in the database


You can do this is psql or dbeaver
```
select count(*) from donorschoose.projects;

select * from donorschoose.projects limit 100;
```

## Workflow tips for loading project data (if it comes in as csvs or even excel)

### Getting raw data into postgres
1. create a ```raw``` schema to store the data you get without much processing
2. Try ohio to load the files directly to postgres. If you get an error, debug :)
2a. If the data file is too large for the RAM you have, process it in chunks
3. Check to make sure the column types make sense and the counts are consistent


### Processing the raw data before analysis
The column types may not be useful for analysis (they may be varchars/strings instead of int or datetime) or you may have one table for each file and you want to merge them into one table.

1. create ```processed``` schema to store the processed data you create (from raw)
2. use sql to change column types, change column names, merge tables, etc.  (create table statements)
3. do not drop rows or columns (ever! but especially not now)
4. check to make sure the counts of rows and columns are still the same









